<a href="https://colab.research.google.com/github/xjdeng/MiniGPT4-image-labeler/blob/main/server_failsafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Base Requirements

In [ ]:
#@title

!git clone https://github.com/oobabooga/text-generation-webui
%cd text-generation-webui
!git checkout 0e6295886d2d0e9309c8223201e8a4bd55e4735c
!git log -n 1 --format=%H
!pip install -r requirements.txt
!git clone https://github.com/Wojtab/minigpt-4-pipeline extensions/multimodal/pipelines/minigpt-4-pipeline
!pip install -r extensions/multimodal/pipelines/minigpt-4-pipeline/requirements.txt
!pip install -U gradio-client flask-cloudflared

# Install GPTQ Requirements

In [ ]:
#@title

!git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
%cd GPTQ-for-LLaMa
!git checkout 53a2aeac935b7eef0e1451eb7b465fea4f318e7a
!git log -n 1 --format=%H
!python setup_cuda.py install
%cd ..
!cp -r GPTQ-for-LLaMa/* .

# Install Model
You can go to https://huggingface.co and pick another Vicuna-based model if you wish and plug the path (without the https://huggingface.co part) below. Note that you may need an A100 Premium Colab GPU for models larger than 13B.

In [ ]:
modelstr = "TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ" #@param {type:"string"}

!python download-model.py $modelstr
modelfolder = modelstr.replace("/","_")

# Run the server
Take note of the URL after "**Starting non-streaming server at public url**".
You may see some error messages but if you don't see a URL after that text above, then please stop and restart the cell below. You'll need to copy the URL of the non-streaming server and paste it into the appropriate fields in the Client notebook. You may need to go to View -> Show/hide output in the next cell.

After you're done running the server, please go up to **Runtime -> Disconnect and delete runtime**.

Otherwise, the server will keep running idle until Google shuts it down and Google may restrict your Colab access in the future if it happens frequently.

In [ ]:
#@title
!python server.py --extensions multimodal --multimodal-pipeline minigpt4-13b --share --chat --loader gptq-for-llama --model_type llama --model $modelfolder --wbits 4 --groupsize 128 --auto-devices --public --public-api --sdp-attention